<a href="https://colab.research.google.com/github/Hataeju/Detection/blob/main/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비

## 기본 라이브러리 Import

In [1]:
# 기본라이브러리 import
from google.colab import drive
import os, json, requests, time  #VirusTotal API결과 분석을 위해

import numpy as np
import pandas as pd


## 라이브러리 추가 설치 및 Import

In [2]:
!pip install pefile

In [3]:
!pip install yara-python

In [4]:
# hashlib은 이제 내장되어있다/
#!pip install\ hashlib

In [5]:
# 추가 라이브러리 import
import hashlib
import pefile
import yara

## 데이터 로드

In [6]:
# 구글 드라이브 mount
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [7]:
# 데이터 파일 위치
colab_path = "gdrive/My Drive/AI보안교육/part2/data/"


# 메타데이터 분석

## 메타데이터 - Hash

In [18]:
# DataFrame 초기화
feature_dict = {'hash':[], 'target':[]}

# Hash를 구하기 위해 hashlib 라이브러리 사용
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb') #바이너리 모드로 읽기
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        feature_dict['hash'].append(hash)
        if dir == "normal":
            feature_dict['target'].append(0)
        else:
            feature_dict['target'].append(1)
df = pd.DataFrame.from_dict(feature_dict)
df = df.set_index('hash')

In [19]:
df

,target
hash,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0


In [20]:
# DataFrame 초기화
feature_dict = {"filename":[], 'md5':[], 'sha256':[], 'sha512':[], 'target':[]}

# Hash를 구하기 위해 hashlib 라이브러리 사용
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb') #바이너리 모드로 읽기
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        md5 = hashlib.md5(data).hexdigest()
        sha256 = hashlib.sha256(data).hexdigest()
        sha512 = hashlib.sha512(data).hexdigest()
        feature_dict['filename'].append(filename)
        feature_dict['md5'].append(md5)
        feature_dict['sha256'].append(sha256)
        feature_dict['sha512'].append(sha512)
        if dir == "normal":
            feature_dict['target'].append(0)
        else:
            feature_dict['target'].append(1)
df2 = pd.DataFrame.from_dict(feature_dict)
df2 = df2.set_index('filename')

In [21]:
df2

,md5,sha256,sha512,target
filename,,,,
cmd.exe,9d59442313565c2e0860b88bf32b2277,d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b...,c489ccb79a65ec41800db7e95d8e4993d7780488355869...,1
net.exe,a63df9a6e9098cc189f2a3efc37600f6,96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17c...,38fb182b83a4d069bc2f3773f6a1b103fce7f0b9d6b6d5...,1
NETSTAT.EXE,2445dece99deedbd701dc6dfe10e648e,5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5...,5815bea5548b47515ce4df76b1cd9618f71a8307aafc7e...,1
systeminfo.exe,4e678d82dde61aaaca201f9636507f9e,f5b4fb293ada97a8c4b92bb380747da1d71336f144332c...,395bf5a1069ac4da7893524337e8e90b4367392c58ded4...,1
powershell.exe,cda48fc75952ad12d99e526d0b6bf70a,908b64b1971a979c7e3e8ce4621945cba84854cb98d763...,31946a5c80088646fe951081e88a7478b804da874236a4...,1
notepad.exe,f1139811bbf61362915958806ad30211,f1d62648ef915d85cb4fc140359e925395d315c70f3566...,5c13fdb1e973d146f6bc33083a023d18bb54d075d7fded...,0
SnippingTool.exe,d57ca86ac22dc057456acc7fde4e492f,6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f...,c0c1145e229d73b939df48b69f7eff80ea3cabb8292349...,0
mspaint.exe,33a61f754d9433d6cc37388ce2711040,f1e3d8851c7b41e3838eae69a95224c92abd9a33117238...,8fe75d720a438ab215f22b5ee873e4476ddd7d9758d2e0...,0
calc.exe,5da8c98136d98dfec4716edd79c7145f,58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba...,6e2b067760ec178cdcc4df04c541ce6940fc2a0cdd36f5...,0


## 메타데이터 - size/name/엔트로피

In [ ]:
from scipy import stats

def get_entropy(filename, start, end):
    f = open(filename, 'rb') #바이트 단위로 read
    information = []
    for i in range(end-start):
        s = f.read(1)
        # 1바이트씩(0~255) 읽어서 정수로 변환: 같은 byte의 발생 frequency 계산을 위해
        information.append(int.from_bytes(s, 'little'))
    value, counts = np.unique(information, return_counts=True)
    # 0~255 각 값의 발생확률을 scipy.stats.entropy를 통해 엔트로피 계산
    probs = counts/sum(counts)
    result = stats.entropy(probs, base=2)
    return result

In [ ]:
# 최종적으로는 파일을 한 번 Load하여 관련 Feature를 전부 추출하겠지만,
# 여기에서는 이론과 병행 학습을 위해 매번 파일을 로드하여 Feature를 하나씩 추가하는 방식을 택한다.

df['size'] = None
df['filename'] = None
df['entropy'] = None

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록




In [ ]:
df

# PE Information

## PE Information - Image File Characteristics

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
hex(pe.FILE_HEADER.Characteristics)

In [ ]:
# Feature로 등록
df['IFC'] = None

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록




In [ ]:
df

## EntryPoint, ImageBase

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
print(hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
print(hex(pe.OPTIONAL_HEADER.ImageBase))

## 섹션 엔트로피 / 사이즈

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
for section in pe.sections:
    if (section.Name).decode('ascii').rstrip('\x00') == '.text':
        start = section.PointerToRawData
        end = section.Misc_VirtualSize
    if (section.Name).decode('ascii').rstrip('\x00') == '.rsrc':
        section_size = section.SizeOfRawData
pe.close()
with open(target_file, 'rb') as f:
    f.seek(start)
    text_data = f.read(end)
text_data

In [ ]:
# Feature로 등록
df['ts_entropy'] = np.NaN
df['rs_size'] = np.NaN

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록




In [ ]:
df

# Static Info

## Yara 기본 동작 원리

In [ ]:
# Text String 기반 탐지
rule = yara.compile(source='rule connection_msg {strings: $a = "connect" condition: $a}')  #C2통신등에 공격자 콘솔에 표시되는 문자열 탐지
matches = rule.match(data='connecting to XXX:')
matches

In [ ]:
# 외부 rule import를 통한 탐지
rule_file = "gdrive/My Drive/AI보안 기술개발 인력양성/악성코드/part2/yara_rules/sample.yar"
yara_rule = open(rule_file, 'r').read()
rule = yara.compile(source=yara_rule)
matches = rule.match(data='connecting to XXX:')  # 메세지 바꿔서 탐지 안되는 것 역시 확인
matches

In [ ]:
# Feature로 등록
df['yara_https'] = None
#rule = yara.compile(source='rule ip_addr {strings: $a = /(\d{1,3}\.){3}\d{1,3}/ condition: $a}')  #IP address
rule = yara.compile(source='rule https {strings: $a = /https:\S+/ condition: $a}') #https
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록




In [ ]:
df

# Dynamic Info

## VT API 기본 사용법

In [ ]:
# Trickbot 샘플로 행위분석 결과 정보 파악
apikey = ""   # 자신의 API Key
hash = "32c58040d3d6ec5305a1a0ebb48ba05aebe3ac2f905a7f152f32fc9170e16711" #SHA-256, SHA-1 또는 MD5

url = "https://www.virustotal.com/api/v3/files/" + hash + "/behaviours?limit=10"
headers = {"Accept": "application/json", "x-apikey": apikey}

response = requests.get(url, headers=headers)

In [ ]:
vt_json = response.json()
vt_json

In [ ]:
# json 최상위 구조 파악
vt_json.keys()

In [ ]:
# 결과를 제공한 sandbox 벤더의 수
vt_json['meta']

In [ ]:
# 쿼리 링크
vt_json['links']

In [ ]:
# 각 벤더마다 동일한 구조로 결과 제공 (여기서는 0~8까지 인덱스. 총 9개 벤더사 결과이므로)
vt_json['data'][0].keys()

In [ ]:
# sandbox 벤더
vt_json['data'][0]['id']

In [ ]:
# 분석 결과물 구조 파악
# 각 벤더마다 제공되는 정보가 모두 다르다. (만약 데이터가 존재한다면 JSON 키값은 동일하다.)
print("{}의 제공정보: {}".format(vt_json['data'][0]['id'].split('_')[1], vt_json['data'][0]['attributes'].keys()))
print("{}의 제공정보: {}".format(vt_json['data'][1]['id'].split('_')[1], vt_json['data'][1]['attributes'].keys()))

In [ ]:
# 그 중 하나의 정보 추출
vt_json['data'][0]['attributes']['files_deleted']

## Dynamic Info - File Activities

In [ ]:
# VT JSON 파일에서 File관련된 Activity만 확인
hash = '908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53'  # 첫번째 샘플에 대해서
load_path = "gdrive/My Drive/AI보안 기술개발 인력양성/악성코드/part2/vt_json/" + hash + ".json"
with open(load_path) as f:
    vt_json = json.load(f)

In [ ]:
vt_json['meta']['count']

In [ ]:
# files_deleted 의 데이터 파악
for vendor_idx in range(vt_json['meta']['count']): #해당 샘플의 경우 vendor 1개
    print("====={}=====".format(vt_json['data'][vendor_idx]['id'].split('_')[1]))
    if 'attributes' in vt_json['data'][vendor_idx].keys():
        for (key, value) in (vt_json['data'][vendor_idx]['attributes'].items()):
            if key in ['files_deleted']:
                print(key, value)


In [ ]:
# 삭제된 파일의 카운트 (각 동적분석 벤더 분석 결과의 평균값).
df['file_deleted_cnt'] = np.nan

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        valid_vendor_cnt = 0
        file_deleted_cnt_total = 0

        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 정보 기록

        load_path = "gdrive/My Drive/AI보안 기술개발 인력양성/악성코드/part2/vt_json/" + hash + ".json"
        with open(load_path) as f:
            vt_json = json.load(f)
        for vendor_idx in range(vt_json['meta']['count']): #해당 샘플의 경우 vendor 1개
            file_deleted_cnt = 0


In [ ]:
df

## Dynamic Info - Registry / Process Activities (실습)